# Cluster Expansion on NMC
<sup>by Alexander S. Tygesen and Jin Hyun Chang</sup>

In this tutorial, we're taking a look at the cathode material known as NMC (Ni-Mn-Co). It is a layered lithium cathode material, and it is a good material for exploration using cluster expansion as due to the large number of possible configurations of Ni, Mn and Co elements. We wiill construct a cluster expansion model for NMC and use the MC model to construct convex-hull curve and simulate the open-circuit voltage (OCV) at various lithation levels for different ratios of Ni, Mn and Co.

A database with pre-calculated energies with varying Ni-Mn-Co ratios and lithation degrees is provided. The structural and energetic information are drawn from DFT calculations, and generating these dataset during the tutorial is impactical as it will take too much time and compuational resources.

## Primitive structure

First, let's take a look at the primitive structure of the layered NMC material. You should see that the material has a layered structure (try repeating the structure for a clearer visual representation). All transition metal (TM) sites are occupied by Co in this cell, but we will allow the TM sites to be occupied by either a Ni, Co or Mn atom.

In [ ]:
from ase.io import read
from ase.visualize import view
prim = read("primitive_nmc.traj")
view(prim)

The structures provided in the database are a $ 2 \times2\times1$ supercell of this primitive cell.

- How many atoms should we expect in the supercell?
- Think about what possible configurations are possible in such a supercell.

## Examining the data

As mentioned previously, we have provided you with a database consisting of randomly generated NMC structures, which is placed in the file ``clease_nmc.db``. It is always a good idea to visualize the data before starting to analyze it.

In [ ]:
!ase db clease_nmc.db

The code below will visualize all of the initial structures in the database. Note that vacancies are marked with an ``X`` atom in ASE.

In [ ]:
from ase.db import connect
con = connect("clease_nmc.db")
view([row.toatoms() for row in con.select(struct_type="initial")])

## Cluster Expansion Settings

First, we need to specify the concentration of each species in the primitive cell. The space group of the structure is 164, and it has 7 basis sites in the primitive cell. One can find these using the ``get_basis`` and ``get_spacegroup`` functions from the ``ase.spacegroup`` module, but we just provide them here:

In [ ]:
import numpy as np
basis = np.array([[0.      , 0.      , 0.      ],
                  [0.666667, 0.333333, 0.333333],
                  [0.333333, 0.666667, 0.166667],
                  [0.      , 0.      , 0.5     ],
                  [0.      , 0.      , 0.239587],
                  [0.666667, 0.333333, 0.093746],
                  [0.666667, 0.333333, 0.57292 ]])
symbols = ["Li", "Li", "Co", "Co", "O", "O", "O"]  # The corresponding symbol for each basis site

# The cell parameters
cell = np.array(prim.get_cell())

From the above, we deduce the following concentration object. A ``grouped_basis`` simply refers to two basis lattices that we consider to be equivalent. In this example, we do not distiguish the two basis of Li, `[0.      , 0.      , 0.      ]` and `[0.666667, 0.333333, 0.333333]`; we treat them to be equivalent by grouping them together using ``grouped_basis``. The same holds for the TM sites (marked with a "Co" symbol) and oxygen sites.

In [ ]:
from clease.settings import Concentration
conc = Concentration(basis_elements=[["Li", "X"], ["Li", "X"],
                                     ["Co", "Mn", "Ni"], ["Co", "Mn", "Ni"],
                                     ["O"], ["O"], ["O"]],
                     grouped_basis=[(0, 1), (2, 3), (4, 5, 6)])

We then construct the settings object based on the spacegroup and the provided basis.

- How large clusters are we going to consider here?

In [ ]:
from clease.settings import CECrystal
db_name = "clease_nmc.db"
settings = CECrystal(conc, db_name=db_name, spacegroup=164, basis=basis, cell=np.array(prim.get_cell()),
                     basis_func_type="binary_linear", max_cluster_dia=[5, 5])

We recommend not adjusting the cluster diameters, as that would require the database to be reconfigured, which may take a while (~5 minutes). Should you want to recalculate the correlation functions, you can do it with the following code (remember to enable)

In [ ]:
if 0:  # Set to 1 to enable
    from clease.tools import reconfigure
    reconfigure(settings, verbose=True)

## Evaluation of the CE model

Since we already have the energies in the database, we do not need to do any energy calculations. Instead, we can simply go straight to fitting as good a CE model as possible.

We encourage you to play around with different settings and schemes, but at least try out the $\ell_1$ and $\ell_2$ regularization schemes. Try and optimize for the best possible CV score.

In [ ]:
%matplotlib notebook
from clease import Evaluate
import clease.plot_post_process as pp

# Set fitting_scheme to "l2" to use l2 regularization instead
fitting_scheme = "l1"
eva = Evaluate(settings=settings, fitting_scheme=fitting_scheme, scoring_scheme='k-fold', nsplits=10)

print("Evaluating alphas")
# scan different values of alpha and return all values of alpha and its corresponding CV scores
alphas, cvs = eva.alpha_CV(alpha_min=1E-7, alpha_max=1.0, num_alpha=50)

# set the alpha value to the one that yields the lowest CV score, and fit data using it.
idx = cvs.argmin()
alpha = alphas[idx]
print(f"Optimized alpha: {alpha:.3e}")

# Plot the cross validation
pp.plot_cv(eva)

eva.set_fitting_scheme(fitting_scheme=fitting_scheme, alpha=alpha)
eva.get_eci()
pp.plot_fit(eva)

# plot convex hull.
pp.plot_convex_hull(eva)

# plot ECI values, an emptry list is passed to ignore_sizes to plot all cluster sizes 
pp.plot_eci(eva)

# save a dictionary containing cluster names and their ECIs
eva.save_eci(fname=f'eci_{fitting_scheme}.json')

## Simulated Annealing

In order to achieve a model of the energies as a function of lithation level, we use an even larger supercell, which allows us a much more fine-grained control. A Monte Carlo (MC) simulation is used in a simulated annealing (SA) to find the lowest energy configuration of a given set of concentrations, and also allows us to explore the temperature dependence of the energies.

In this example, we will explore NMC with a 6:2:2 ratio between Ni, Mn and Co ($\mathrm{LiNi_{0.6} Mn_{0.2} Co_{0.2} O_2}$), which is one of the stable phases known experimentally. One could also have explored these configurations using the CE model, to try and identify the stable mixing ratios.

First step is to set up a model template structure. We use a $8\times8\times2$ supercell of the primitive cell, which totals 1536 atoms. It is definitely possible to use a larger cell, but it would be more demanding on the computational resources, so we restrict ourselves to a reasonable size of $8\times8\times2$ supercell for demonstration. 

In [ ]:
supercell = settings.prim_cell * (8, 8, 2)
view(supercell)

We then also need to set the concentrations of the respective sublattices. As mentioned previously, we are going to be calculating 622 NMC.

In [ ]:
template = supercell.copy()
mask = template.symbols == "Co"  # Location of the TM atoms
N = sum(template.symbols == "Co")  # Number of TM sites

N_Co = int(0.2 * N)  # 20% Co
N_Mn = int(0.2 * N)  # 20% Mn
N_Ni = N - N_Co - N_Mn   # ~60% Ni (remainder)

print(f"Ni: {N_Ni}, Mn: {N_Mn}, Co: {N_Co}")

new_symbols = N_Co * ["Co"] + N_Mn * ["Mn"] + N_Ni * ["Ni"]
template.symbols[mask] = new_symbols
view(template)

Verify that we indeed have approximately 60% Ni, 20% Co and 20% Mn in our working supercell by visualization. What is the degree of lithiation in this template?

Next, we need construct the CLEASE calculator object, which is going to be evaluating the energies during the MC simulations. For this, we need the ECI values which you fitted earlier.

In [ ]:
import json
from clease.calculator import attach_calculator

fitting_scheme = "l1"
filename = f'eci_{fitting_scheme}.json'
with open(filename) as file:
    eci = json.load(file)
print("Loaded ECI. Attaching calculator...")
    
atoms = attach_calculator(settings, template, eci=eci)
print("Done!")

Finally, we are ready to run our SA. Set the number of swaps (``n_steps``), and adjust the temperature ranges as you see fit. You may need to reduce the number of steps if and swaps if the simulation is taking too long, or increase it if the simulation is not good enough. 

In [ ]:
from clease.montecarlo import Montecarlo
from clease.montecarlo.constraints import ConstrainSwapByBasis
from clease.montecarlo.observers import LowestEnergyStructure
from tqdm.notebook import tqdm

def run_mc(atoms, t_max=30_000, t_min=1, t_steps=10):
    # Temperatures in log-space
    temps = np.logspace(np.log10(t_max), np.log10(t_min), t_steps)

    n_steps = len(atoms) * 3  # How many swaps are we going to do per temperature?

    mc = Montecarlo(atoms, temps[0])

    # We need to dis-allow sublattices from mixing
    cnst = ConstrainSwapByBasis(atoms, settings.index_by_basis)
    mc.generator.add_constraint(cnst)

    # Keep track of the lowest energy structure
    emin_obs = LowestEnergyStructure(atoms)
    mc.attach(emin_obs)

    # We track the average energy and other thermodynamic quantities as a function of temp
    results = []

    with tqdm(temps) as t:
        for temp in t:
            t.set_description(f"Temp: {temp:.1f} K")
            mc.T = temp
            mc.run(n_steps)
            res = mc.get_thermodynamic_quantities()
            results.append(res)
    # Return the minimum energy atoms and the thermodynamic quantities we collected
    return emin_obs.emin_atoms, results

emin_atoms, results = run_mc(atoms)  # Execute the MC run

Let's take a look at configuration which gave us the lowest energy during the entire run.

In [ ]:
emin = emin_atoms.get_potential_energy()
print(f"Lowest energy during SA: {emin:.24} eV")
view(emin_atoms)

We can construct a Pandas data frame from the MC results, and get an overview of the structural evolution as a function of temperature:

In [ ]:
import pandas as pd
df = pd.DataFrame(results)
df

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
x = df['temperature']
y = df['energy'] / len(atoms)  # Average energy per atom
ax.plot(x, y)
ax.invert_xaxis()
ax.set_xscale("log")
ax.set_ylabel("Average Energy (eV/atom)")
ax.set_xlabel("Temperature (K)")

## Simulating the lithiation degrees

In the previous part, we had 100% lithation. We now move to modelling different degrees of lithation. To do this, we reuse the same template from previously, but now introduce vacancies (marked as `X` atoms) on the lithium sites.

In [ ]:
def make_delithiated(li_frac: float):
    """This function constructs a new atoms object from the template with a given degree of lithiation."""
    mask = template.symbols == "Li"   # Locate all Li sites
    tot_sites = sum(mask)
    n_li = int(li_frac * tot_sites)
    n_vac = tot_sites - n_li
    
    new_symbols = n_li * ["Li"] + n_vac * ["X"]
    atoms = template.copy()
    atoms.symbols[mask] = new_symbols
    return atoms

Let's run the MC at various lithation degrees:

In [ ]:
all_results = []
emin_atoms = []
for x in np.linspace(1, 0, 20):
    print(f"Lithation degree: {x*100:.2f} %")
    atoms = make_delithiated(x)
    # Remember to attach a new calculator to the atoms object
    atoms = attach_calculator(settings, atoms, eci=eci)
    # Execute the MC
    ats, results = run_mc(atoms, t_steps=20)
    # Collect and store the results
    for result in results:
        # Add a custom key in the result dictionary with the current lithation degree
        # This makes processing and plotting the data later a little easier.
        result['lithiation'] = x
    all_results.extend(results)
    emin_atoms.append(ats)  

In [ ]:
df_li = pd.DataFrame(all_results)
df_li.to_json("nmc_lithiation_data.json")  # Save the data frame to a file, so we can recover it later
df_li

We can also visualize all of the minimum energy structures at each calculated concentration level. What kind of patterns do you see?

In [ ]:
view(emin_atoms)

Finally, let's look at the convex hull of the structures, and study the temperature dependence.

In [ ]:
import numpy as np
from scipy.spatial import ConvexHull
from ase.formula import Formula
from collections import namedtuple
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, interp2d

# Energy per atom in BCC Li
e_li_bulk = -1.954936

x = df_li["lithiation"]
y = df_li["temperature"]
# Energy per formula unit, we know we always have 2 Oxygen per unit cell
z = df_li["energy"] / (sum(atoms.symbols=="O") / 2)  

# 2D Interpolate the lithation degree and temperature to the energy, i.e.
#  en_interp(x_li, temp) = energy
# linearly interpolates between each datapoint
en_interp = interp2d(x, y, z)

def get_formation_energy(x, temp):
    """Formation energy:
    E_x - x * E_1 - x * E_0,
    where x is the degree of lithation, E_1 is the fully lithiated structure,
    and E_0 is completely delithiated
    """
    val = en_interp(x, temp) - x * en_interp(1, temp) - (1 - x) * en_interp(0, temp)
    return float(val[0])  # "val" is a a list with 1 element

def calculate_all_energies(temp, npts=100):
    """Calculate the formation energies at all lithiation degrees from 0 to 1."""
    x_range = np.linspace(0, 1, npts)
    en = np.array([get_formation_energy(x, temp) for x in x_range])
    return x_range, en

def get_lower_convex_simplices(temp):
      
    lithiation, energies = calculate_all_energies(temp)
    ch_points = list(zip(lithiation, energies))
    
    hull = ConvexHull(ch_points)
    
    def is_lower_hull(simplex):
        return all(ch_points[i][1] <= 0 for i in simplex)

    #print([simplex for simplex in hull.simplices])
    #print([is_lower_hull(simplex) for simplex in hull.simplices])

    simplices = sorted(filter(is_lower_hull, hull.simplices), key=lambda simplex: min(lithiation[simplex]))
    simplices = np.array(simplices)
    return simplices

def get_ocv(temp):
    x, y = calculate_all_energies(temp)
    simplices = get_lower_convex_simplices(temp)

    # Recalculate the voltage on a new grid
    grid = np.linspace(0, 1, 500)
    dx = grid[1] - grid[0]

    all_points = sorted(set(s for simplex in simplices for s in simplex))

    # Interpolate the energy onto the energies on the hull
    f = interp1d(x[all_points], en_interp(x[all_points], temp), kind="linear")
    G = f(grid)
    dG = np.gradient(G, dx)

    # The voltage
    v = - (dG - e_li_bulk)
    return grid, v

def plot_ch(temp, ax):
    x, y = calculate_all_energies(temp)
    simplices = get_lower_convex_simplices(temp)
    
    ax.plot(x, y, 'o')
    # Plot simplices
    for ii, simplex in enumerate(simplices):
        x_cnv = x[simplex]
        y_cnv = y[simplex]
        if ii == 0:
            # Only make label on first plot
            label1 = "Lower hull"
            label2 = "Stable points on hull"
        else:
            label1, label2 = None, None
        ax.plot(x_cnv, y_cnv, 'k-', label=label1)
        ax.plot(x_cnv, y_cnv, 'ro', label=label2)
    ax.set_ylabel('E$_{form}$ (eV)')
    ax.legend()
    ax.set_xlabel("$x$ in Li$_x$(NMC)O$_2$")

def plot_ocv(tempp, ax):
    grid, v = get_ocv(temp)
    ax.plot(grid, v)
    ax.set_xlabel("$x$ in Li$_x$(NMC)O$_2$")
    ax.set_ylabel("Voltage (V)")

Let's plot the convex hull and OCV

In [ ]:
temp = 300  # Set the desired temperature
fig, (ax0, ax1) = plt.subplots(nrows=2, ncols=1, sharex=True)
plot_ch(temp, ax0)
plot_ocv(temp, ax1)
ax0.set_title(f"Temperature: {temp:.1f} K")

How does your results compare to the experimentally determined OCV from Appl. Sci. 2019, 9, 3671: [doi:10.3390/app9183671](https://doi.org/10.3390/app9183671)

<img src="nmc_622_ocv.png" style="width: 700px;"/>